In [53]:
# Import Python Tools:
from IPython.display import display, Latex
import pandas as pd
import numpy as np
from numpy import sqrt, pi, average, cos, sin, tan, arcsin, arccos, arctan, deg2rad, rad2deg
# import sectionproperties as shape

<div style="page-break-after: always;"></div>

# Seismic Load - Equivalent Lateral Force Method

### Seismic Parameters

In [54]:
site_class = "default" #A, B, C, D, E, F, or default
S_s = 1.5
S_1 = 0.683

F_a = 1.2
F_v = 1.7

In [55]:
S_MS = S_s * F_a
S_M1 = S_1 * F_v
round(S_MS, 3), round(S_M1, 3)

(1.8, 1.161)

In [56]:
S_DS = 2/3 * S_MS
S_D1 = 2/3 * S_M1
T_L = 16 #sec
I_e = 1.5

R = 8
Ω_0 = 2.5
C_d = 5
ρ = 1.3

### Seismic Response Coefficient, $C_s$

<img src="./attachments/Table_12.8-2.png" width=500>

In [57]:
hn = 50 #ft
Ct = 0.03
x = 0.75
T = Ct*hn**x

f"Approximate fundamental period, Ta = {round(T, 3)} sec"

'Approximate fundamental period, Ta = 0.564 sec'

In [58]:
Cs_calc = S_DS / (R/I_e)
print(f"Prelim Cs = {round(Cs, 4)}")

if T <= T_L:
	Cs = min(Cs_calc, S_D1 / (T*R/I_e))
elif T > T_L:
	Cs = min(Cs_calc, S_D1 / (T**2*R/I_e))

print(f"Adjusted Cs = {round(Cs, 4)}")

Prelim Cs = 0.225
Adjusted Cs = 0.225


### Effective Seismic Weight

<img src="./attachments/effective seismic weight.png" width=500>

In [59]:
roof_DL = 15 #psf
wall_DL = 15 #psf

roof_area = 120/2 * 350 #sf
wall_perimeter = 350 #ft

W = (
	roof_DL*roof_area \
	+ wall_DL*wall_perimeter \
) / 1000 #kips

f"total seismic weight = {round(W, 2)} kips"

'total seismic weight = 320.25 kips'

### Base Shear

In [60]:
V = Cs * W #kips
f"Base Shear, V = {round(V, 2)} kips"

'Base Shear, V = 72.06 kips'

### Vertical Distribution

<img src="./attachments/vertical distribution.png" width=500>

In [61]:
h_roof = hn #ft
k = 1.0

Cv_roof = 1.0

F_roof = Cv_roof * V #kips
f"force on roof = {round(F_roof, 2)} kips"

'force on roof = 72.06 kips'

### Diaphragm Forces

In [62]:
F_px_calc = F_roof #kips
F_design_roof = F_px_calc #kips
f"Diaphragm Force = {round(F_design_roof, 2)} kips"

'Diaphragm Force = 72.06 kips'

### Story Drift

In [63]:
def seismic_drift(δ_xe):
	δ_x = C_d * δ_xe / I_e
	return δ_x

In [64]:
elastic_story_drifts = [0.1, 0.3, 5] #in (from external calculation)

seismic_story_drifts = []
for δ_xe in elastic_story_drifts:
	δ_x = seismic_drift(δ_xe)
	seismic_story_drifts.append(δ_x)
	
f"Seismic Drifts = {seismic_story_drifts} in"

'Seismic Drifts = [0.3333333333333333, 1.0, 16.666666666666668] in'